In [6]:
import requests
from bs4 import BeautifulSoup

papers = []

In [7]:
def parse_doi(doi):
    a = doi[16:]
    if(a[0] == 'b'):
        a = a[5:]
    return a

def extract_papers(page):
    filtered_content_type = ['extended-abstract', 'research-article', 'Work in Progress', 'poster', 'short-paper']
    soap = BeautifulSoup(page.text, 'html.parser')

    for entry in soap.find_all("li", attrs={"class": "issue-item-container"}):
        title = entry.find('h5', attrs={'class': 'issue-item__title'}).text
        content_type = entry.find('div', attrs={'class': 'issue-heading'})
        # doi = parse_doi(entry.find('a')['href'])
        doi = entry.find("a", attrs={'class': 'issue-item__doi'})
        # years = entry.find('div', attrs={'class': ['bookPubDate', 'simple-tooltip__block--b']})
        if(content_type.text in filtered_content_type and doi != None):
            papers.append({
            "title": title,
            "doi": parse_doi(doi.get('href'))})

In [8]:
for i in range(3):
    print(f"Get page {i}")
    page = requests.get(f"https://dl.acm.org/action/doSearch?AllField=%22human-ai+interaction%22&startPage={i}&pageSize=500")
    extract_papers(page)

Get page 0
Get page 1
Get page 2


In [12]:
def get_data_from_semantic_scholar(doi):
    base_url = "https://api.semanticscholar.org/v1/paper/"
    url = f"{base_url}{doi}"
    headers = {"Accept": "application/json"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        abstract = data.get('abstract')
        year = data.get('year')
        references = [ref['doi'] for ref in data.get('references')]
        
        if abstract:
            return [abstract, year, references]
        else:
            return ["no abstract"]
    
    except requests.exceptions.RequestException as e:
        print(e)
        return ["error"]

In [10]:
import pandas as pd

df = pd.DataFrame()

for i, entry in enumerate(papers):
    fetched = get_data_from_semantic_scholar(entry["doi"])
    print(f"fetching #{i}:", entry["title"])
    if fetched[0] != "error" and len(fetched) == 3:
        df = df._append({
            "title": entry["title"],
            "year": fetched[1],
            "doi": entry["doi"],
            "abstract": fetched[0],
            "references": fetched[2]
        }, ignore_index=True)

fetching #0: HAID: Human-AI Interaction for Dementia Care
fetching #1: Theory of Mind in Human-AI Interaction
fetching #2: Assessing Human-AI Interaction Early through Factorial Surveys: A Study on the Guidelines for Human-AI Interaction
fetching #3: Spiritual AI: Exploring the Possibilities of a Human-AI Interaction Beyond Productive Goals
fetching #4: Development and translation of human-AI interaction models into working prototypes for clinical decision-making
fetching #5: Exploration of Explainable AI for Trust Development on Human-AI Interaction
fetching #6: Re-examining User Burden in Human-AI Interaction: Focusing on a Domain-Specific Approach
fetching #7: A Systematic Review on Fostering Appropriate Trust in Human-AI Interaction: Trends, Opportunities and Challenges
fetching #8: Measuring User Experience Inclusivity in Human-AI Interaction via Five User Problem-Solving Styles
fetching #9: Human-AI interaction: intermittent, continuous, and proactive
fetching #10: Designing for 

In [11]:
# save df to csv
df.to_csv("fetch.csv")